# Типы сервисов

- Пример предназначен для демонстрации работы по получению списка типов сервисов. 
- Особенность заключается в маппинге или адаптации типов сервисов из БД к типам сервисов `blocksnet`.

Нужны только те типы сервисов, которые:
- Имеют `infrastructure_type`.
- Или привязаны к `soc_value_id`.
- При этом существуют в `blocksnet`.

## 1. Вспомогательный код

In [1]:
import prostor.fetchers.misc as misc
from prostor.adapters import adapt_service_types

def get_service_types():
  service_types = misc.get_service_types()
  return adapt_service_types(service_types)

## 2. Демонстрация

In [4]:
service_types = get_service_types()
service_types.head()

,name,infrastructure_type,infrastructure_weight,social_values
service_type_id,,,,
1,park,basic,0.2,[8]
5,beach,comfort,0.5,None
21,kindergarten,basic,0.2,[2]
22,school,basic,0.2,"[2, 4]"
27,university,comfort,0.2,"[4, 10, 17]"


In [5]:
service_types.to_pickle('./data/service_types.pickle')